In [26]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

rFolder = "C:/Users/janin/Downloads/"
dataFolder = "D:/Repositories/Global-COVID-Surveillance/data/"
cleanedFolder = dataFolder + "cleaned/"

In [27]:
# Countries and Regions

european_countries = [
    'Albania','Andorra','Austria','Belarus','Belgium','Bosnia & Herzegovina','Bulgaria',
    'Croatia','Czech Republic','Denmark','Estonia','Finland','France',
    'Germany','Greece','Greenland','Hungary','Iceland','Ireland','Isle of Man','Italy',
    'Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Moldova','Monaco','Montenegro',
    'Netherlands','Norway','Poland','Portugal','Romania',
    'San Marino','Serbia','Slovakia','Slovenia','Spain','Sweden','Switzerland',
    'Ukraine','United Kingdom','Vatican City'
]

north_american_countries = ["Canada","United States"]

carribean_countries = [
    "Antigua & Barbuda","Aruba","Bahamas","Barbados","Bermuda","British Virgin Islands",
    "Cayman Islands","Cuba","Curacao","Dominica","Dominican Republic","Grenada",
    "Haiti","Jamaica","Puerto Rico","St. Kitts & Nevis","St. Lucia","St. Vincent & Grenadines",
    "Sint Maarten","Trinidad & Tobago","Turks and Caicos Islands","United States Virgin Islands"
]
central_south_america_countries = [
    'Argentina','Belize','Bolivia','Brazil','Chile','Colombia','Costa Rica',
    'Ecuador','El Salvador','Guatemala','Guyana','Honduras',
    'Mexico','Nicaragua','Panama','Paraguay','Peru','Suriname','Uruguay','Venezuela'
]
latin_american_countries = carribean_countries + central_south_america_countries
american_countries = north_american_countries + latin_american_countries

south_asia_countries = [
    "Afghanistan","Bangladesh","Bhutan","India","Maldives","Nepal","Pakistan","Sri Lanka"
]
central_asian_countries = [
    'Armenia','Azerbaijan','Cyprus','Faeroe Islands','Georgia','Gibraltar','Kazakhstan','Kosovo','Kyrgyzstan',
    'North Macedonia','Russia','Tajikistan','Turkey','Turkmenistan','Uzbekistan'
]
east_asian_countries = [
    "Brunei","Cambodia","China","Indonesia","Japan","Laos","Malaysia","Mongolia","Myanmar","Niue","North Korea","Philippines",
    "Singapore","South Korea","Taiwan","Thailand","Timor","Vietnam"
]
pacific_countries = [
    "Australia","Cook Islands","Fiji","French Polynesia","Guam","Kiribati",
    "Marshall Islands","Micronesia","Nauru","New Caledonia","New Zealand",
    "Northern Mariana Islands","Palau","Papua New Guinea","Samoa","Solomon Islands","Tonga","Tuvalu","Vanuatu"
]
east_asia_and_pacific_countries = east_asian_countries + pacific_countries

middle_eastern_countries = [
    "Bahrain","Iran","Iraq","Israel","Jordan","Kuwait","Lebanon","Oman","Qatar","Saudi Arabia","Syria",
    "United Arab Emirates","Yemen"
]
north_african_countries = [
    "Algeria","Djibouti","Egypt","Libya","Morocco","Tunisia","Western Sahara"
]
middle_east_and_north_africa_countries = middle_eastern_countries + north_african_countries

sub_saharan_african_countries = [
    "Angola","Benin","Botswana","Burkina Faso","Burundi",
    "Cabo Verde","Cameroon","Central African Republic","Chad","Comoros","Côte d’Ivoire",
    "Democratic Republic of Congo","Equatorial Guinea","Eritrea","Ethiopia",
    "Gabon","Gambia","Ghana","Guinea","Guinea-Bissau","Kenya","Lesotho","Liberia",
    "Madagascar","Malawi","Mali","Mauritania","Mauritius","Mozambique",
    "Namibia","Niger","Nigeria","Republic of the Congo","Rwanda",
    "São Tomé and Príncipe","Senegal","Seychelles","Sierra Leone",
    "Somalia","South Africa","South Sudan","Sudan","Swaziland",
    "Tanzania","Togo","Uganda","Zambia","Zimbabwe"
]

unincorporated_disputed_territories = [
    "American Samoa", "Anguilla","Caribbean Netherlands","Channel Islands","Curaçao",
    "Falkland Islands","French Guiana","Guadeloupe","Hong Kong"
]

country_lists = [
    central_asian_countries,
    east_asia_and_pacific_countries,
    european_countries,
    latin_american_countries,
    middle_east_and_north_africa_countries,
    north_american_countries,
    sub_saharan_african_countries,
    south_asia_countries
]

all_countries = []
for country_list in country_lists:
    all_countries = all_countries + country_list
all_countries.sort()

regions = [
    'Central Asia',
    'East Asia and Pacific',
    'Europe',
    'Latin America',
    'Middle East and North Africa',
    'North America',
    'South Asia',
    'Sub-Saharan Africa',
    'Territory'
]

countries_by_region = {
    'Central Asia': central_asian_countries,
    'Europe': european_countries,
    'Latin America': latin_american_countries,
    'South Asia': south_asia_countries,
    'Sub-Saharan Africa': sub_saharan_african_countries,
    'Middle East and North Africa': middle_east_and_north_africa_countries,
    'East Asia and Pacific': east_asia_and_pacific_countries,
    'North America': north_american_countries,
    'Territory': unincorporated_disputed_territories
}

country_conversions = {
    "Antigua & Barbuda": ["Antigua and Barbuda"],
    "Bahamas": ["Bahamas, The"],
    "Bosnia & Herzegovina": ["Bosnia and Herzegovina"],
    "Brunei": ["Brunei Darussalam"],
    "Cabo Verde": ["Cape Verde"],
    "Côte d’Ivoire": ["Cote d'Ivoire","Cote dIvoire"],
    "Czech Republic": ["Czechia","Czech Republic (Czechia)"],
    "Democratic Republic of Congo": ["Congo - Kinshasa"],
    "Egypt": ["Egypt, Arab Rep."],
    "Faeroe Islands": ["Faroe Islands"],
    "Gambia": ["Gambia, The"],
    "Hong Kong" : ["Hong Kong SAR China"],
    "Iran": ["Iran, Islamic Rep."],
    "Kyrgyzstan": ["Kyrgyz Republic"],
    "Laos": ["Lao PDR"],
    "Micronesia": ["Micronesia, Fed. Sts."],
    "Myanmar": ["Myanmar (Burma)","Burma"],
    "North Macedonia": ["Macedonia"],
    "State of Palestine": ["Palestinian Territories"],
    "Republic of the Congo": ["Congo - Brazzaville"],
    "Russia": ["Russian Federation"],
    "São Tomé and Príncipe": ["Sao Tome and Principe","Sao Tome & Príncipe","São Tomé & Príncipe"],
    "Sint Maarten": ["Sint Maarten (Dutch part)"],
    "Slovakia": ["Slovak Republic"],
    "St. Kitts & Nevis": ["Saint Kitts and Nevis"],
    "St. Lucia": ["Saint Lucia"],
    "St. Vincent & Grenadines": ["Saint Vincent and the Grenadines"],
    "Swaziland": ["Eswatini"],
    "Syria": ["Syrian Arab Republic"],
    "Timor": ["Timor-Leste"],
    "Trinidad & Tobago": ["Trinidad and Tobago"],
    "Vatican City": ["Holy See"],
    "Yemen": ["Yemen, Rep."],
    "" : ["nan"]
}

census_regions = {
    0: {"name" : "United States",
        "states" : ["United States"]},
    1: {"name" : "Northeast",
        "states" :["Connecticut", "Maine", "New Hampshire", "Vermont", "Massachusetts", 
                   "Rhode Island", "New Jersey", "New York", "Pennsylvania"]},
    3: {"name" : "South",
        "states" : ["Maryland", "Delaware", "West Virginia", "Virginia", "Kentucky", 
                    "Tennessee", "North Carolina", "South Carolina", "Georgia", "Florida", 
                    "Alabama", "Mississippi", "Arkansas", "Louisiana", "Oklahoma", "Texas", 
                    "District of Columbia", "Puerto Rico"]},
    2: {"name" : "Midwest",
        "states" : ["North Dakota", "South Dakota", "Nebraska", "Kansas", "Missouri", "Iowa", 
                    "Minnesota", "Wisconsin", "Illinois", "Michigan", "Indiana", "Ohio"]},
    4: {"name" : "West",
        "states" : ["Washington", "Idaho", "Montana", "Wyoming", "Oregon", "California", "Nevada", 
                    "Utah", "Colorado", "Arizona", "New Mexico", "Alaska", "Hawaii"]}
}

us_states = [
    'Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia',
    'Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
    'Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska',
    'Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota',
    'Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas',
    'Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming'
]
canada_provinces = [
    'Alberta','British Columbia','Manitoba','New Brunswick','Newfoundland and Labrador','Northwest Territories',
    'Nova Scotia','Ontario','Prince Edward Island','Quebec','Saskatchewan','Yukon'
]
states_and_provinces = us_states + canada_provinces

In [43]:
us_populations_file = "D:/Repositories/Global-COVID-Surveillance/data/raw/demographics/US Population.xlsx"
us_populations = pd.read_excel(us_populations_file)
us_populations["Region"] = "North America"
us_populations = us_populations.drop(["Population Source"],axis=1)
us_populations.head()

,Country,State/Province,Census Region,Population,Population 100K,Female,Male,Pct Male,Pct Female,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+,Region
0,United States,Alabama,South,4889347,48.89347,2531653,2357694,0.482210,0.517790,56901,58290,59073,59799,60294,59568,58599,59537,60023,60241,60897,63083,62906,61883,61729,61740,61799,61924,62938,64125,63587,64201,63943,63719,63922,65079,65208,67027,69478,68758,64852,61469,59980,59615,60721,58941,59921,60346,60696,62200,58159,57993,57852,55498,58174,57008,58838,61959,65460,64750,60738,59494,59786,61321,65925,66906,66695,67073,67308,68221,65605,65211,65365,63117,62042,59584,56766,54694,52697,51707,50567,49884,51612,37091,36845,35441,36173,30575,27572,26053,23977,22580,19594,18222,16660,91543,237456,608466,631898,642187,589780,615279,657543,501447,256847,0.011638,0.048566,0.124447,0.129240,0.131344,0.120626,0.125841,0.134485,0.102559,0.052532,0.018723,North America
1,United States,Alaska,West,712114,7.12114,347065,365049,0.512627,0.487373,9978,10012,10186,10509,10395,10414,10303,10286,10436,10157,9976,10016,9887,9509,9678,9488,9410,9343,8518,7525,8088,8617,9132,9252,9900,10318,10693,11456,11576,11552,10946,10809,10460,10822,10799,10303,10452,9962,9667,9685,8865,8589,8533,7954,8295,7827,7962,8230,8696,9086,8428,8197,8330,8595,9128,9426,9493,9636,9731,9894,9373,9168,9161,8614,8800,8210,7734,7220,6655,6442,5978,5621,5482,4013,3945,3665,3459,2950,2677,2327,1971,1784,1586,1411,1277,7181,41102,100662,89273,109431,92305,84479,93296,61300,23107,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084,North America
2,United States,Arizona,West,7259090,72.59090,3658425,3600665,0.496022,0.503978,81929,83065,85726,88192,90876,90858,90405,90319,91313,90797,92572,96605,96951,95849,95076,94742,91832,93119,97724,99334,95779,96720,97447,99492,100066,101835,102053,105016,106612,105885,99473,96012,93463,94419,94495,91292,93193,93126,92593,94337,87480,85369,85480,83038,86427,83705,82925,85342,90167,90646,85428,82772,81434,81904,87011,88220,88617,88864,90069,90909,88041,87973,87664,85106,85274,83374,81125,78896,77728,77376,76474,76787,81299,59741,58899,57521,58174,50088,45100,41784,38116,35128,31252,28331,25703,145737,347859,930745,966255,999263,892335,851334,880737,751699,411197,0.011286,0.047920,0.128218,0.133110,0.137657,0.122927,0.117278,0.121329,0.103553,0.056646,0.020076,North America
3,United States,Arkansas,South,3012542,30.12542,1535409,1477133,0.490328,0.509672,36355,37006,37572,38610,38921,38404,37924,38827,38633,38959,38941,40404,41015,40146,39960,39598,39485,39395,38933,39714,40206,40211,40323,39367,38992,39539,39518,40912,42271,41927,39361,38289,37446,37354,37897,37550,38010,38198,38328,39332,36427,36037,35410,34319,35486,34449,34938,36485,37966,38136,35481,34716,34766,35572,38680,39707,39698,39360,39525,39697,38668,38092,37865,36917,36430,35478,34176,32682,31736,30888,30427,30017,31554,22864,23007,22169,22217,19362,17669,16670,14936,13764,12330,11253,10771,59912,152109,393213,396224,394514,369097,361189,385959,302829,161141,0.012068,0.050492,0.130525,0.131525,0.130957,0.122520,0.119895,0.128117,0.100523,0.053490,0.019888,North America
4,United States,California,West,39356141,393.56141,19843586,19512555,0.495794,0.504206,462589,462713,477322,485894,495198,493458,494221,493396,504330,493445,492283,511109,512662,507455,505628,503712,501846,497188,515261,501692,493088,497749,512251,533604,557011,576604,588951,613288,640318,640758,611094,595453,577581,574306,575253,556953,559625,555569,542576,549416,510476,499294,496070,486794,500853,483161,483781,493411,516222,535475,504901,490446,479620,481731,5065

In [ ]:
# Read starter

pgmm = pd.read_excel(dataFolder + "Starter.xlsx")
print(pgmm.columns)
pgmm.head()

# Read R results

pgmm_files = [
    'Central Asia',
    'East Asia and Pacific',
    'Europe',
    'Latin America',
    'Middle East and North Africa',
    'United States',
    'Canada',
    'South Asia',
    'Sub-Saharan Africa'
]

for r in pgmm_files:
    df = pd.read_csv(rFolder + r + " -Results .csv")
    df = df.drop(["Unnamed: 0"], axis=1)
    if ((r == "Canada") or (r == "United States")):
        df["Region"] = "North America"
        df.rename(columns = {"V2": "State/Province"}, inplace=True)
        df["Country"] = r
        df["Level"] = df["State/Province"].apply(lambda x: "Country" if (x == "Region") else "State/Province")
        df["State/Province"] = df["State/Province"].apply(lambda x: "" if (x == "Region") else x)
    else:
        df["Region"] = r
        df.rename(columns = {"V2": "Country"}, inplace=True)
        df["Level"] = df["Country"].apply(lambda x: "Region" if (x == "Region") else "Country")
        df["Country"] = df["Country"].apply(lambda x: "" if (x == "Region") else x)
        df["State/Province"] = ""
    df.rename(columns = {
        "V1": "Statistics Excel Date",
        "V3": 'Statistics Cases Weekly', 
        "V4": 'Statistics Total Cases',
        "V5": 'Statistics Cases Weekly Average', 
        "V6": 'Infection Rate', 
        "V7": 'Statistics Deaths Weekly', 
        "V8": 'Statistics Total Deaths',
        "V9": 'Statistics Cases Weekly Rate', 
        "V10": 'Statistics Deaths Weekly Rate', 
        "V11": 'Speed', 
        "V12": 'Acceleration', 
        "V13": 'Jerk',
        "V14": '1-Day Persistence', 
        "V15": '7-Day Persistence',
        "Level": "Statistics Level",
        "Region": "Statistics Region",
        "Country": "Statistics Country",
        "State/Province": "Statistics State/Province"
    }, inplace=True)
    pgmm = pd.concat([pgmm, df], ignore_index=True, sort=False)

# Create Time Variables
pgmm["Statistics Excel Date"] = pgmm["Statistics Excel Date"].astype(int)
pgmm["Statistics Date"] = pgmm["Statistics Excel Date"].apply(lambda x: datetime.fromordinal(datetime(1900, 1, 1).toordinal() + x - 2))
pgmm["Statistics Week"] = pgmm["Statistics Date"].apply(lambda x: x.strftime('%Y%V'))

In [39]:
# Output cleaned dataset
pgmm_order = [
    'Statistics Level', 'Statistics Region', 'Statistics Country', 'Statistics State/Province', 'Statistics Excel Date', 'Statistics Date', 'Statistics Week',
    'Speed', 'Acceleration', 'Jerk', '7-Day Persistence',
    '1-Day Persistence', 'Infection Rate',
    'Statistics Total Cases', 'Statistics Cases Weekly', 'Statistics Cases Weekly Rate',
    'Statistics Cases Weekly Average',
    'Statistics Total Deaths', 'Statistics Deaths Weekly', 'Statistics Deaths Weekly Rate',
]
pgmm = pgmm[pgmm_order]
print(pgmm.columns)
pgmm_file = cleanedFolder + "pgmm.xlsx"
print(pgmm_file)
pgmm.to_excel(pgmm_file, index=False)

pgmm.loc[pgmm["Statistics State/Province"]=="Alberta"].head(100)

Index(['Level', 'Region', 'Country', 'State/Province', 'Excel Date'], dtype='object')
Index(['Statistics Level', 'Statistics Region', 'Statistics Country',
       'Statistics State/Province', 'Statistics Excel Date', 'Statistics Date',
       'Statistics Week', 'Speed', 'Acceleration', 'Jerk', '7-Day Persistence',
       '1-Day Persistence', 'Infection Rate', 'Statistics Total Cases',
       'Statistics Cases Weekly', 'Statistics Cases Weekly Rate',
       'Statistics Cases Weekly Average', 'Statistics Total Deaths',
       'Statistics Deaths Weekly', 'Statistics Deaths Weekly Rate'],
      dtype='object')
D:/Repositories/Global-COVID-Surveillance/data/cleaned/pgmm.xlsx


,Statistics Level,Statistics Region,Statistics Country,Statistics State/Province,Statistics Excel Date,Statistics Date,Statistics Week,Speed,Acceleration,Jerk,7-Day Persistence,1-Day Persistence,Infection Rate,Statistics Total Cases,Statistics Cases Weekly,Statistics Cases Weekly Rate,Statistics Cases Weekly Average,Statistics Total Deaths,Statistics Deaths Weekly,Statistics Deaths Weekly Rate
1106,State/Province,North America,Canada,Alberta,44105,2020-10-01,202040,3.376072,0.048460,0.016153,2.536975,0.367677,3.912367,18227.0,173.0,1.142857,149.285714,269.0,2.0,0.045230
1107,State/Province,North America,Canada,Alberta,44112,2020-10-08,202041,4.791115,0.617062,0.649369,2.894257,0.461203,8.231800,19710.0,364.0,2.000000,211.857143,283.0,2.0,0.045230
1108,State/Province,North America,Canada,Alberta,44119,2020-10-15,202042,5.572942,-0.387683,-0.710752,4.107352,0.658606,5.518020,21435.0,244.0,0.714286,246.428571,288.0,1.0,0.022615
1109,State/Province,North America,Canada,Alberta,44126,2020-10-22,202043,7.708428,0.591216,0.064614,4.777601,0.786401,9.656535,23821.0,427.0,1.142857,340.857143,296.0,0.0,0.000000
1110,State/Province,North America,Canada,Alberta,44133,2020-10-29,202044,10.380210,0.161535,0.148612,6.608322,1.129090,10.787277,27034.0,477.0,3.142857,459.000000,318.0,5.0,0.113074
1111,State/Province,North America,Canada,Alberta,44140,2020-11-05,202045,11.097423,-1.541040,-1.977183,8.898801,1.396459,0.000000,30469.0,0.0,3.571429,490.714286,343.0,0.0,0.000000
1112,State/Province,North America,Canada,Alberta,44147,2020-11-12,202046,19.151535,4.949418,6.710145,10.633509,2.597392,34.645929,36397.0,1532.0,7.142857,846.857143,393.0,17.0,0.384452
